In [6]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import re
from tqdm import  tqdm
import os
import collections
import gc
from nltk.probability import ConditionalFreqDist, ConditionalProbDist, MLEProbDist, FreqDist
import math

In [7]:
file_path = "/kaggle/input/amharic-corpus-general/GPAC.txt"
n = 2

In [9]:
os.path.exists(file_path)
if os.path.exists(file_path):
    print(os.stat(file_path).st_size / (1024 * 1024))
else:
    print(os.path.exists(file_path))

1013.849326133728


# 1) N-gram language model

## 1.1 Create n-grams for n=1, 2, 3, 4. You can show sample prints.

In [10]:
# Downloading necessary NLTK data
nltk.download('punkt')

def process_chunk(chunk, n):
    tokens = word_tokenize(re.sub(r'\W+', ' ', chunk))
    return list(ngrams(tokens, n))

def read_and_process_in_chunks(file_path, n, chunk_size=1024*1024):
    with open(file_path, 'r') as file:
        while True:
            chunk = file.read(chunk_size)
            if not chunk:
                break
            yield process_chunk(chunk, n)


all_ngrams = []
for ngrams_chunk in read_and_process_in_chunks(file_path, n):
    all_ngrams.extend(ngrams_chunk)
    gc.collect()


[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


In [11]:
print(len(all_ngrams))

77759851


In [12]:
print(all_ngrams[:10])

[('ምን', 'መሰላችሁ'), ('መሰላችሁ', 'አንባቢያን'), ('አንባቢያን', 'ኢትዮጵያ'), ('ኢትዮጵያ', 'በተደጋጋሚ'), ('በተደጋጋሚ', 'ጥሪው'), ('ጥሪው', 'ደርሷት'), ('ደርሷት', 'ልትታደመው'), ('ልትታደመው', 'ያልቻለችው'), ('ያልቻለችው', 'የአለም'), ('የአለም', 'የእግር')]


## 1.2 Calculate probabilities of n-grams and find the top 10 most likely n-grams for all n.

In [15]:
# Create a conditional frequency distribution of Amharic n-grams
def create_freq_dist(ngrams):
    cfreq = ConditionalFreqDist((tuple(ngram[:-1]), ngram[-1]) for ngram in ngrams)
    return cfreq

# Create a conditional probability distribution using maximum likelihood estimation
def create_prob_dist(cfreq):
    cprob = ConditionalProbDist(cfreq, MLEProbDist)
    return cprob


probabilites = []

cfreq = create_freq_dist(all_ngrams)
cprob = create_prob_dist(cfreq)
probabilities = cprob
gc.collect()

    

0

In [ ]:
def get_top_ngrams(cpd, n):
    top_ngrams = []
    for context in cpd.conditions():
        for word in cpd[context].samples():
            probability = cpd[context].prob(word)
            top_ngrams.append((context + (word,), probability))
    
    top_ngrams.sort(key=lambda x: x[1], reverse=True)
    top_ngrams = top_ngrams[:10]
    return top_ngrams

In [ ]:
top_ngrams = get_top_ngrams(probabilities, 10)
print(top_ngrams)

## 1.3 What is the probability of the sentence. "ኢትዮጵያ ታሪካዊ ሀገር ናት ". You can also try more sentences.

In [ ]:
sequence = ("ኢትዮጵያ" ,"ታሪካዊ" ,"ሀገር", "ናት")
probability = 1.0
bigram_counts = cfreq


# Calculate the probability of the sequence
for i in range(1, len(sequence)):
    prev_word = sequence[i-1]
    current_word = sequence[i]
    count = bigram_counts[prev_word][current_word]
    total_count = bigram_counts[prev_word].N()
    conditional_probdist = MLEProbDist(bigram_counts[prev_word])
    conditional_prob = conditional_probdist.prob(current_word)
    probability *= conditional_prob

    
print("Probability:", probability)

## 1.4 Generate random sentences using n-grams; explain what happens as n increases, based on your output.

In [ ]:
# Generate a sentence using the n-gram model
def generate_sentence(cprob, max_length=20):
    sentence = []
    context = cprob.conditions()[0]
    while len(sentence) < max_length:
        word = cprob[context].generate()
        sentence.append(word)
        context = context[1:] + (word,)
        if word == '።':
            break
    return ' '.join(sentence)

In [ ]:
# Generate a sentence using the n-gram model
generated_sentence = generate_sentence(probabilities)
print(generated_sentence)

# 2 Evaluate these Language Models Using Intrinsic Evaluation Method

In [16]:
def calculate_perplexity(cpdist, test_set):
    log_probabilities = [-math.log(cpdist[seq[:-1]].prob(seq[-1],)) for seq in test_set]
    perplexity = math.exp(sum(log_probabilities) / len(log_probabilities))
    return perplexity

test_set = all_ngrams[:30]

perplexity = calculate_perplexity(probabilities, test_set)
print("Perplexity:", perplexity)


Perplexity: 191.34567451384368
